### Figure 1: Predicted images

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import metpy.plots.ctables
from utils.data import load_tc_data
from utils.plot import make_cmap

# TODO: because i'm using imshow the lats need to be flipped - normally would be fine but because lats are flipped and imshow doesn't know this
# the TCs are plotted spinning the wrong way

sns.set_style("white")
sns.set_palette(sns.color_palette("Paired"))
sns.set_palette(sns.color_palette("Set2"))

In [2]:
# filters gen 256, filters disc 512, 1280000 samples, extra two residual blocks in disc might have generated rainbands and better structure.
# undoing changes above and doing 640000 sambpled with 720 filters disc made power spectra much much better

In [3]:
# define cmap
precip_cmap,precip_norm = make_cmap()

In [4]:
def regrid(array):
        hr_array = np.zeros((100,100))
        for i in range(10):
                for j in range(10):
                        i1 = i*10
                        i2 = (i+1)*10
                        j1 = j*10
                        j2 = (j+1)*10
                        hr_array[i1:i2,j1:j2] = array[i,j]
        return hr_array

In [5]:
def plot_predictions(pred_0,pred_1,pred_2,pred_3,real,storms,range_,plot='save'):
	if plot == 'save':
			width_ratios = [0.2, 1,1,1,1,1,1,1,1,0.1]
			fig, axes = plt.subplots(n, m+2, figsize=(5*m+6, 5*n), sharey=True,gridspec_kw={'width_ratios': width_ratios})
	else:
			print('show')
			fig, axes = plt.subplots(n, m+2, figsize=(2*m+2, 2*n), sharey=True)

	pal = sns.cubehelix_palette(10, rot=-.25, light=.7,as_cmap=True)

	for i in range(m+1):
		for j in range(n):
			axes[j,i].set(xticklabels=[])
			axes[j,i].set(yticklabels=[])
			axes[j,0].spines['bottom'].set_visible(False)
			axes[j,0].spines['top'].set_visible(False)
			axes[j,0].spines['left'].set_visible(False)
			axes[j,0].spines['right'].set_visible(False)
			axes[j,-1].spines['bottom'].set_visible(False)
			axes[j,-1].spines['top'].set_visible(False)
			axes[j,-1].spines['left'].set_visible(False)
			axes[j,-1].spines['right'].set_visible(False)

	for i in range(m):
		j = 0
		
		storm = storms[i]
		# storm_og = [6385,15028,11023,12976][i]
		# storm_og= [15436,13922,11023,11173][i]
		storm_og = [13126,13530,13265,12965,2564,11976,15766,2384][i]
		# norm = colors.Normalize(*range_)
		norm = precip_norm
		
		axes[j,i+1].imshow(pred_0[storm], interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
		axes[j,i+1].invert_yaxis()
		axes[j+1,i+1].imshow(pred_1[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
		axes[j+1,i+1].invert_yaxis()
		axes[j+2,i+1].imshow(pred_2[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
		axes[j+2,i+1].invert_yaxis()
		axes[j+3,i+1].imshow(pred_3[storm_og], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
		axes[j+3,i+1].invert_yaxis()
		im = axes[j+4,i+1].imshow(real[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
		axes[j+4,i+1].invert_yaxis()


		# axes[j,i+1].set(xticklabels=[])
		# axes[j,i+1].set(yticklabels=[])
		# axes[j+1,i+1].set(xticklabels=[])
		# axes[j+1,i+1].set(yticklabels=[])
		# axes[j+2,i+1].set(xticklabels=[])
		# axes[j+2,i+1].set(yticklabels=[])
		# axes[j+3,i+1].set(xticklabels=[])
		# axes[j+3,i+1].set(yticklabels=[])
		# axes[j+4,i+1].set(xticklabels=[])
		# axes[j+4,i+1].set(yticklabels=[])
		# axes[j,0].set(xticklabels=[])
		# axes[j,0].set(yticklabels=[])
		# axes[j+1,0].set(xticklabels=[])
		# axes[j+1,0].set(yticklabels=[])
		# axes[j+2,0].set(xticklabels=[])
		# axes[j+2,0].set(yticklabels=[])
		# axes[j+3,0].set(xticklabels=[])
		# axes[j+3,0].set(yticklabels=[])
		# axes[j+4,0].set(xticklabels=[])
		# axes[j+4,0].set(yticklabels=[])
		


		axes[j,0].set_ylabel('Scalar WGAN',rotation=0,horizontalalignment='right',fontsize=48,labelpad=4,fontweight='bold',color='#56a1d7')
		axes[j+1,0].set_ylabel('Modular WGAN',rotation=0,horizontalalignment='right',fontsize=48,labelpad=4,fontweight='bold',color='#56a1d7')
		axes[j+2,0].set_ylabel('Modular WGAN raw',rotation=0,horizontalalignment='right',fontsize=48,labelpad=4,fontweight='bold',color='#56a1d7')
		axes[j+3,0].set_ylabel('WGAN',rotation=0,horizontalalignment='right',fontsize=48,labelpad=4,fontweight='bold',color='#56a1d7')
		axes[j+4,0].set_ylabel('MSWEP',horizontalalignment='right',rotation=0,fontsize=48,labelpad=4,fontweight='bold',color='#56a1d7')
		

		# axes[4,j].set_xlabel('Maria (2017; NA)',fontsize=24,labelpad=5)
		# axes[4,j+1].set_xlabel('Haiyan (2013; WP)',fontsize=24,labelpad=5)
		# axes[4,j+2].set_xlabel('Barbara (1995; EP)',fontsize=24,labelpad=5)
		# axes[4,j+3].set_xlabel('Vayu (2019; NI)',fontsize=24,labelpad=5)
		# axes[4,j+4].set_xlabel('Ursula (1998; SP)',fontsize=24,labelpad=5)
		# axes[4,j+5].set_xlabel('Brendan (1985; WP)',fontsize=24,labelpad=5)

	fig.subplots_adjust(right=0.8)
	# cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
	cbar_ax = fig.add_axes([1, 0.031, 0.03, 0.936])
	levels=[0, 1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 70, 100, 150]
	cb = fig.colorbar(im, cax=cbar_ax,shrink=0.01,ticks=levels)
	cb.ax.tick_params(labelsize=36)
	cb.set_label('mm / 3hr', fontsize=48)
	fig.tight_layout()

	if plot == 'save':
			plt.savefig('figure_1_new_new_v36.png',bbox_inches='tight',dpi=600)
			plt.clf()
	else:
			plt.show()
	

In [6]:
# load data
real,inputs,pred,meta = load_tc_data(set='validation',results='ke_tracks')
real_2,inputs_2,pred_2,meta_2,imput_og,pred_og,meta_og = load_tc_data(set='validation',results='kh_tracks')
modular_pred_2 = np.load('/user/home/al18709/work/gan_predictions_20/modular_part2_lowres_predictions_pred-opt_modular_part2_raw.npy')
pred_mraw = np.load('/user/home/al18709/work/gan_predictions_20/validation_pred-opt_modular_part2_raw.npy')
meta_og = pd.read_csv('/user/work/al18709/tc_data_mswep_40/valid_meta.csv')
meta_valid = pd.read_csv('/user/work/al18709/tc_data_flipped/valid_meta.csv')

In [7]:
storms = [8453,9845,1375,3014,3487,3738]
i = 3487
storms = [9900,1075,3111,202]
i = 202
storms = [976,54,5076,3478,4004,4105,5002,6010]
i = 55
sid = meta.loc[i]['sid']
print(sid)
centre_lat = meta.loc[i]['centre_lat']
centre_lon = meta.loc[i]['centre_lon']

new_i = meta_og.where((meta_og['sid']==sid) & (meta_og['centre_lat']==centre_lat) & (meta_og['centre_lon']==centre_lon),False)
new_i = meta_og.index[(meta_og['sid']==sid) & (meta_og['centre_lat']==centre_lat) & (meta_og['centre_lon']==centre_lon)].to_list()
print(new_i) 

# [6385,15028,11023,12976]

# print(meta.loc[i])
# print(meta_og.loc[i])
# print(meta_2.loc[i])
# print(meta_og.loc[new_i])
# # [_,6409,15174,11108,13078]

# print(meta_og.shape)
# print(meta.shape)
# print(meta_2.shape)
# print(pred_og.shape)

# meta_test = pd.read_csv('/user/work/al18709/tc_data_mswep_40/valid_meta.csv')
# print(meta_test.shape)
# [15436,13922,2322,11173]
# [13126,13530,13265,12965,2564,11976,15766,2384]

1979002S04179
[13530]


In [8]:

plot = 'save'
n = 5
m = 4

n = 5
m = 8

# make colours
range_ = (0, 20)
storms = [4077, 4078, 4079, 4080, 4081, 4082]

# maria 8453 (8452 for unet) 8463
# haiyan 9820, 9828,  9845, 9840 9815, 9816, 9822 9809
# barb 1347 1375 1399
# georges 4082

storms = [8453,9845,1375,4128,5358,747]
storms = [8453,9845,1375,3014,3487,3757] 
storms = [8453,9845,1375,3014,3487,3738]
storms = [9845,1375,3014,3487]
# storm_og = [6385,15028,11023,12976][i]
# storms_extremes = [4082,5349,763]
# storms = [8453,9845,1375,3014]
storms = [9900,1075,3014,202]
storms = [976,55,5076,3478,4004,4105,5002,6010]
plot_predictions(pred[:,:,:,2],modular_pred_2[:,:,:,2],pred_mraw[:,:,:,0],pred_og[:,:,:,0],real,storms,range_)


/tmp/ipykernel_91428/4055331942.py:90: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


<Figure size 4600x2500 with 0 Axes>

In [9]:
print(inputs.shape)

(15930, 10, 10, 6)
